Function
-------
(Urban & tMin) - (Rural & tMin)

(Urban & tAvg) - (Rural & tAvg)


Stats
-----------------
 - June, July, August 2004-2013
 - Average summer (June to Aug daily temp)

In [1]:
import pickle
with open('climatecentral/climatecentralbase2.pickle', 'rb') as fout:
    sampleplaces = pickle.load(fout)
    
print len(sampleplaces.values())

100


In [2]:
import numpy as np
import pandas as pd

outputdict = []

for s in sampleplaces.values():
    urbanuse = s['station'].get('urbanclosest', None)
    if not urbanuse:
        print "skipping", s['usgsplacename']
        continue
    try:
        urbandf = pd.DataFrame(urbanuse['results'], columns=['measure', 'year', 'value'])
    except:
        print "no results found"
        continue
    try:
        ruraluse = s['station']['ruralclosest']
        if len(ruraluse) == 0 or not ruraluse:
            print "skipping rural", s['usgsplacename']
            continue
        ruraldfs = []
        for r in ruraluse:
            tdf = pd.DataFrame(r['results'], columns=['measure', 'year', 'value'])
            tdf.set_index(['measure', 'year'])
            ruraldfs.append(tdf)
        df_concat = pd.concat(ruraldfs)
    #     print ruraldfs

        ruralresult = pd.DataFrame(df_concat.groupby(['measure','year'])['value'].mean())


        mergeddf =  urbandf.merge(ruralresult.reset_index(),on=['measure','year'],how='left')
    #     print mergeddf[['value_x', 'value_y']].sub(axis=1)
        mergeddf['uhi'] = mergeddf['value_x'] - mergeddf['value_y']
        outputdict.append({
                'placename': s['usgsplacename'],
                'uhitmin': mergeddf[mergeddf['measure']=='tmin']['uhi'].mean(),
                'uhitavg': mergeddf[mergeddf['measure']=='tavg']['uhi'].mean(),
                'uhitmax': mergeddf[mergeddf['measure']=='tmax']['uhi'].mean()
            })
    except Exception,e:
        print "Error", e
        
#     print mergeddf[['value_x']].sub(mergeddf['value_y'], axis=0)
#     for year in range(2004, 2014):
#         for measure in ['tmin', 'tmax', 'tavg']


skipping rural Denver
Error 'NoneType' object has no attribute '__getitem__'
Error 'NoneType' object has no attribute '__getitem__'
skipping rural Reno
skipping Lubbock
no results found
Error 'NoneType' object has no attribute '__getitem__'


In [3]:
import csv

with open('climatecentral/stationresults.csv', 'w') as fin:
    writer = csv.DictWriter(fin, fieldnames=['placename', 'uhitmin', 'uhitmax', 'uhitavg'])
    writer.writeheader()
    for row in outputdict:
        writer.writerow(row)